Trying to analyze the gt points' distribution with respect to predicted points location

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
%matplotlib qt

import scipy.stats

import torchvision.datasets as dsets
import torchvision.transforms as transforms
# from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader
from torch.autograd import Variable

import my_model
from utilities import MyTrainDataSet, MyTestDataSet, load_data_2, load_test_data, min_max_scaling, min_max_scaling_radius, normalize_one, construct_train_valid_tensor, construct_test_tensor, show_statistic

# get training data

In [2]:
input_train = np.load("input_tensor.npy")
gt_label = np.load("gt_label_tensor.npy")
pd_label = np.load("pd_label_tensor.npy")

input_valid_train = np.load("input_valid_tensor.npy")
gt_valid_label = np.load("gt_valid_label_tensor.npy")
pd_valid_label = np.load("pd_valid_label_tensor.npy")

# convert to sperical coordinate

In [3]:
def getDistanceThetaPhi(xyz): # N x 3 -> N x 2, xyz_tensor must be two dimension
    size = xyz.shape[0]
    dtp = np.zeros([size, 3])
    for i in range(size):
        x = xyz[i, 0]
        y = xyz[i, 1]
        z = xyz[i, 2]
        # print(z)
        d = math.sqrt(x*x + y*y + z*z)
        theta = math.atan(math.sqrt(x*x + y*y)/z)/math.pi*180
        phi = math.atan(y/x)/math.pi*180
        
        if (d >= 2.0):
            d= 2.0;
        elif (d <= 1.0):
            d= 1.0;
        
        if (theta < 0):
            theta = 180 + theta;
        
        if (phi > 0):
            if (x > 0):
                phi = phi;
            else:
                phi = 180 + phi;
        else:
            if (x < 0):
                phi = 180 + phi;
            else:
                phi = 360 + phi;
        
        
        dtp[i, 0] = d
        dtp[i, 1] = theta
        dtp[i, 2] = phi
    
    return dtp   

In [ ]:
# test the function's validity
pd_dthetaphi_label_tensor = getDistanceThetaPhi(pd_label_tensor)
gt_dthetaphi_label_tensor = getDistanceThetaPhi(gt_label_tensor)

test_set_x_1, test_set_y_1, test_set_z_1 = load_test_data('test/test_1.csv')
test_set_x_2, test_set_y_2, test_set_z_2 = load_test_data('test/test_2.csv')
test_set_x_3, test_set_y_3, test_set_z_3 = load_test_data('test/test_3.csv')
test_set_x_4, test_set_y_4, test_set_z_4 = load_test_data('test/test_4.csv')
test_set_x_5, test_set_y_5, test_set_z_5 = load_test_data('test/test_5.csv')

test_set_x_1_direct, test_set_y_1_direct, test_set_z_1_direct = load_test_data('test/test_1_direct.csv')
test_set_x_2_direct, test_set_y_2_direct, test_set_z_2_direct = load_test_data('test/test_2_direct.csv')
test_set_x_3_direct, test_set_y_3_direct, test_set_z_3_direct = load_test_data('test/test_3_direct.csv')
test_set_x_4_direct, test_set_y_4_direct, test_set_z_4_direct = load_test_data('test/test_4_direct.csv')
test_set_x_5_direct, test_set_y_5_direct, test_set_z_5_direct = load_test_data('test/test_5_direct.csv')

input_xyz = np.zeros([400, 3])
for i in range(400):
    input_xyz[i, 0] = test_set_x_1[i]
    input_xyz[i, 1] = test_set_y_1[i]
    input_xyz[i, 2] = test_set_z_1[i]
    
input_dthetaphi_tensor = getDistanceThetaPhi(input_xyz)

x_diff = [i - j for i, j in zip(test_set_x_1_direct, input_dthetaphi_tensor[:,0].tolist())]
y_diff = [i - j for i, j in zip(test_set_y_1_direct, input_dthetaphi_tensor[:,1].tolist())]
z_diff = [i - j for i, j in zip(test_set_z_1_direct, input_dthetaphi_tensor[:,2].tolist())]

# plt.plot(test_set_x_1_direct, label='d')
# plt.plot(test_set_y_1_direct, label='theta')
plt.plot(test_set_z_1_direct, label='pi')
plt.legend()

In [4]:
# construct 2d theta phi training tensor
gt_dthetaphi_label = getDistanceThetaPhi(gt_label)
pd_dthetaphi_label = getDistanceThetaPhi(pd_label)
gt_thetaphi_label_tensor = torch.Tensor(gt_dthetaphi_label[:,1:])
pd_thetaphi_label_tensor = torch.Tensor(pd_dthetaphi_label[:,1:])
print("gt_thetaphi_label_tensor shape:", gt_thetaphi_label_tensor.shape)
print("pd_thetaphi_label_tensor shape:", pd_thetaphi_label_tensor.shape)

# construct 2d theta phi valid tensor
gt_dthetaphi_valid_label = getDistanceThetaPhi(gt_valid_label)
pd_dthetaphi_valid_label = getDistanceThetaPhi(pd_valid_label)
gt_thetaphi_valid_label_tensor = torch.Tensor(gt_dthetaphi_valid_label[:,1:])
pd_thetaphi_valid_label_tensor = torch.Tensor(pd_dthetaphi_valid_label[:,1:])
print("gt_thetaphi_valid_label_tensor shape:", gt_thetaphi_valid_label_tensor.shape)
print("pd_thetaphi_valid_label_tensor shape:", pd_thetaphi_valid_label_tensor.shape)

gt_thetaphi_label_tensor shape: torch.Size([14985, 2])
pd_thetaphi_label_tensor shape: torch.Size([14985, 2])
gt_thetaphi_valid_label_tensor shape: torch.Size([397, 2])
pd_thetaphi_valid_label_tensor shape: torch.Size([397, 2])


In [5]:
gt_thetaphi_label_tensor = torch.matmul(gt_thetaphi_label_tensor, torch.Tensor([[1/180, 0],[0, 1/360]]))
pd_thetaphi_label_tensor = torch.matmul(pd_thetaphi_label_tensor, torch.Tensor([[1/180, 0],[0, 1/360]]))

gt_thetaphi_valid_label_tensor = torch.matmul(gt_thetaphi_valid_label_tensor, torch.Tensor([[1/180, 0],[0, 1/360]]))
pd_thetaphi_valid_label_tensor = torch.matmul(pd_thetaphi_valid_label_tensor, torch.Tensor([[1/180, 0],[0, 1/360]]))

In [6]:
class MDN(nn.Module):
    def __init__(self, n_input, n_hidden, n_gaussians):
        super(MDN, self).__init__()
        self.l_h = nn.Sequential(
            nn.Linear(n_input, n_hidden),
            nn.Tanh()
        )
        self.l_pi = nn.Linear(n_hidden, n_gaussians)
        
        self.l_mu_theta = nn.Linear(n_hidden, n_gaussians)
        self.l_sigma_theta = nn.Linear(n_hidden, n_gaussians)
        
        self.l_mu_phi = nn.Linear(n_hidden, n_gaussians)
        self.l_sigma_phi = nn.Linear(n_hidden, n_gaussians)
        
        
        self.l_correlation_theta_phi = nn.Sequential(
            nn.Linear(n_hidden, n_gaussians),
            nn.Tanh()
        )
        
    def forward(self, x):
        h = self.l_h(x)
        # print("h", h.shape)
        # print("h[0]", h[0, :])
        
        pi = F.softmax(self.l_pi(h), -1)
        
        # print("pi", pi.shape)
        # print("pi[0]", pi[0, :])
        mu_theta = self.l_mu_theta(h)
        # print("mu_theta", pi.shape)
        mu_phi = self.l_mu_phi(h)
        # print("mu_phi", pi.shape)
        
        # use exp to ensure positive range
        sigma_theta = torch.exp(self.l_sigma_theta(h))
        # print("sigma_theta", sigma_theta.shape)
        sigma_phi = torch.exp(self.l_sigma_phi(h))
        # print("sigma_phi", sigma_phi.shape)

        # use tanh to ensoure range of (-1, 1)
        correlation_theta_phi = self.l_correlation_theta_phi(h)
        # print("correlation_y_z", pi.shape)
        # print("correlation_y_z[0]", correlation_y_z[0, :])
        
        return pi, mu_theta, mu_phi, sigma_theta, sigma_phi, correlation_theta_phi


In [17]:
def mdn_loss_fn(y, pi, mu_theta, mu_phi, sigma_theta, sigma_phi, correlation_theta_phi):
    size = y.shape[0]
    n_gaussians = pi.shape[1]
    '''
    print("sample size: ", size)
    print("num of gaus: ", n_gaussians)
    print("mu_theta size: ", mu_theta.shape)
    print("mu_phi size: ", mu_phi.shape)
    print("sigma_theta size: ", sigma_theta.shape)
    print("sigma_phi size: ", sigma_phi.shape)
    print("correlation_theta_phi size: ", correlation_theta_phi.shape)
    '''
    # build mean matrix
    mean = torch.stack((mu_theta, mu_phi), dim=2)
    # build covariance matrix with standard bivariate normal distribution
    cov = torch.zeros([size, n_gaussians, 2, 2])
    cov[:, :, 0, 0] = sigma_theta**2
    cov[:, :, 1, 1] = sigma_phi**2
    cov[:, :, 1, 0] = correlation_theta_phi*sigma_theta*sigma_phi
    cov[:, :, 0, 1] = correlation_theta_phi*sigma_theta*sigma_phi

    
    '''
    print("sigma_theta**2 size: ", (sigma_theta**2).shape)
    print("sigma_phi**2 size: ", (sigma_phi**2).shape)
    print("correlation_theta_phi*sigma_theta*sigma_phi size: ", (correlation_theta_phi*sigma_theta*sigma_phi).shape)
    print("mean size:", mean.shape)
    print("cov size:", cov.shape)
    print("correlation_theta_phi:", correlation_theta_phi.min())
    '''
    # mean: N x n_gaussian x 2
    # cov: N x 5 x 2 x 2
    m = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)
    # expand y to N x n_gaussian x 2
    y = y.unsqueeze(1).repeat(1, n_gaussians, 1)
    
    # print("y size", y.shape) # 14985 x 5 x 2
    
    # y: [x, y, z]; row: x, y, z; column: N, number of samples
    # y: N x 3
    # loss: N x 1
    # print(y_normal.is_cuda)
    likelihood = torch.exp(m.log_prob(y))
    
    # print("likelihood shape: ", likelihood.shape) # 14985 x 5
    # print("pi shape: ", pi.shape) # 14985 x 5
    loss = torch.sum(likelihood * pi, dim=1) # 14985 X 1
    # print(likelihood.max())
    '''
    for i in range(loss.shape[0]):
        if loss[i] > 1.5:
            print("loss: ", loss[i])
            print("likelihood: ", likelihood[i])
            print("mu array  : ", pi[i])
            # print("likelihood shape: ", likelihood.shape) # 14985 x 5
    '''
    loss = -torch.log(loss)
    return torch.mean(loss)

In [49]:
num_gaussians = 7
PATH = "model/with_early_stop_after_400_without_normalization/model_ng_7.pt"
PATH_LOG = "model/with_early_stop_after_400_without_normalization/model_ng_7.txt"

In [50]:
model_mdn = MDN(2, n_hidden=20, n_gaussians=num_gaussians)
#if torch.cuda.is_available():
#    model.cuda()

optimizer = torch.optim.Adam(model_mdn.parameters())

In [51]:
train_loss = []
valid_loss = []

min_loss = 100.0 # starts from large number
min_epoch = 0
early_stop_epoch_size = 800

f = open(PATH_LOG, "w")

for epoch in range(4000):
    optimizer.zero_grad()
    pi, mu_theta, mu_phi, sigma_theta, sigma_phi, correlation_theta_phi = model_mdn(pd_thetaphi_label_tensor)
    # print("xx", correlation_theta_phi)
    loss = mdn_loss_fn(gt_thetaphi_label_tensor, pi, mu_theta, mu_phi, sigma_theta, sigma_phi, correlation_theta_phi)
    loss.backward()
    optimizer.step()
    train_loss.append(loss.data.item())
    
    # Calculating validation data loss
    total_valid_loss = 0.0
    valid_pred = []
    pi_valid, mu_theta_valid, mu_phi_valid, sigma_theta_valid, sigma_phi_valid, correlation_theta_phi_valid = model_mdn(pd_thetaphi_valid_label_tensor)
    loss_valid = mdn_loss_fn(gt_thetaphi_valid_label_tensor,
                       pi_valid, mu_theta_valid,
                       mu_phi_valid,
                       sigma_theta_valid,
                       sigma_phi_valid,
                       correlation_theta_phi_valid)
    valid_loss.append(loss_valid.data.item())
    
    if epoch % 1 == 0:
        print("-------epoch ", epoch, "-------")
        # print(loss.data.tolist())
        print("epoch: {}, train_loss: {}, valid_loss: {}".format(epoch,
                                                                 loss.data.item(),
                                                                 loss_valid.data.item()))
    # check early stop condition
    if (min_loss > loss_valid.data.item()):
        min_loss = loss_valid.data.item()
        min_epoch = epoch
        torch.save(model_mdn.state_dict(), PATH)
    if (epoch - min_epoch > early_stop_epoch_size):
        log = "early stop at: " + str(epoch) + "; optimal epoch at: " + str(min_epoch)
        print(log)
        f.write(log)
        f.close()
        break


-------epoch  0 -------
epoch: 0, train_loss: 2.2311391830444336, valid_loss: 2.123826026916504
-------epoch  1 -------
epoch: 1, train_loss: 2.202756404876709, valid_loss: 2.0981380939483643
-------epoch  2 -------
epoch: 2, train_loss: 2.1746091842651367, valid_loss: 2.0725746154785156
-------epoch  3 -------
epoch: 3, train_loss: 2.146663188934326, valid_loss: 2.0471138954162598
-------epoch  4 -------
epoch: 4, train_loss: 2.1188881397247314, valid_loss: 2.021713972091675
-------epoch  5 -------
epoch: 5, train_loss: 2.091248035430908, valid_loss: 1.9963469505310059
-------epoch  6 -------
epoch: 6, train_loss: 2.0637121200561523, valid_loss: 1.9710006713867188
-------epoch  7 -------
epoch: 7, train_loss: 2.0362534523010254, valid_loss: 1.9456671476364136
-------epoch  8 -------
epoch: 8, train_loss: 2.0088517665863037, valid_loss: 1.9203394651412964
-------epoch  9 -------
epoch: 9, train_loss: 1.9814907312393188, valid_loss: 1.8950083255767822
-------epoch  10 -------
epoch: 10,

-------epoch  83 -------
epoch: 83, train_loss: -0.2533907890319824, valid_loss: -0.08021481335163116
-------epoch  84 -------
epoch: 84, train_loss: -0.28836166858673096, valid_loss: -0.11169707775115967
-------epoch  85 -------
epoch: 85, train_loss: -0.32430291175842285, valid_loss: -0.14434991776943207
-------epoch  86 -------
epoch: 86, train_loss: -0.36128196120262146, valid_loss: -0.17814111709594727
-------epoch  87 -------
epoch: 87, train_loss: -0.3993763029575348, valid_loss: -0.21304626762866974
-------epoch  88 -------
epoch: 88, train_loss: -0.4386758506298065, valid_loss: -0.2490670382976532
-------epoch  89 -------
epoch: 89, train_loss: -0.4792761504650116, valid_loss: -0.28624671697616577
-------epoch  90 -------
epoch: 90, train_loss: -0.5212711691856384, valid_loss: -0.3246854543685913
-------epoch  91 -------
epoch: 91, train_loss: -0.5647561550140381, valid_loss: -0.3645469546318054
-------epoch  92 -------
epoch: 92, train_loss: -0.6098455786705017, valid_loss: -

-------epoch  165 -------
epoch: 165, train_loss: -4.532167911529541, valid_loss: -3.213017225265503
-------epoch  166 -------
epoch: 166, train_loss: -4.51732063293457, valid_loss: -3.1043920516967773
-------epoch  167 -------
epoch: 167, train_loss: -4.487064361572266, valid_loss: -3.2019803524017334
-------epoch  168 -------
epoch: 168, train_loss: -4.477367877960205, valid_loss: -3.118603229522705
-------epoch  169 -------
epoch: 169, train_loss: -4.528889179229736, valid_loss: -3.2393438816070557
-------epoch  170 -------
epoch: 170, train_loss: -4.606052875518799, valid_loss: -3.2347397804260254
-------epoch  171 -------
epoch: 171, train_loss: -4.644625186920166, valid_loss: -3.1811492443084717
-------epoch  172 -------
epoch: 172, train_loss: -4.629603385925293, valid_loss: -3.237459897994995
-------epoch  173 -------
epoch: 173, train_loss: -4.588708877563477, valid_loss: -3.0992093086242676
-------epoch  174 -------
epoch: 174, train_loss: -4.5652384757995605, valid_loss: -3.

-------epoch  247 -------
epoch: 247, train_loss: -5.145107269287109, valid_loss: -3.262385606765747
-------epoch  248 -------
epoch: 248, train_loss: -5.158353805541992, valid_loss: -3.139995813369751
-------epoch  249 -------
epoch: 249, train_loss: -5.1934123039245605, valid_loss: -3.2677385807037354
-------epoch  250 -------
epoch: 250, train_loss: -5.2350382804870605, valid_loss: -3.214097261428833
-------epoch  251 -------
epoch: 251, train_loss: -5.267380237579346, valid_loss: -3.2365241050720215
-------epoch  252 -------
epoch: 252, train_loss: -5.279935836791992, valid_loss: -3.2590138912200928
-------epoch  253 -------
epoch: 253, train_loss: -5.274507522583008, valid_loss: -3.1794135570526123
-------epoch  254 -------
epoch: 254, train_loss: -5.25889778137207, valid_loss: -3.2671120166778564
-------epoch  255 -------
epoch: 255, train_loss: -5.240471839904785, valid_loss: -3.1344664096832275
-------epoch  256 -------
epoch: 256, train_loss: -5.226588726043701, valid_loss: -3

-------epoch  329 -------
epoch: 329, train_loss: -5.566189289093018, valid_loss: -3.269015073776245
-------epoch  330 -------
epoch: 330, train_loss: -5.574483394622803, valid_loss: -3.194591760635376
-------epoch  331 -------
epoch: 331, train_loss: -5.5821661949157715, valid_loss: -3.263514995574951
-------epoch  332 -------
epoch: 332, train_loss: -5.588509559631348, valid_loss: -3.1978373527526855
-------epoch  333 -------
epoch: 333, train_loss: -5.593327522277832, valid_loss: -3.2613449096679688
-------epoch  334 -------
epoch: 334, train_loss: -5.596205711364746, valid_loss: -3.1885857582092285
-------epoch  335 -------
epoch: 335, train_loss: -5.596465110778809, valid_loss: -3.2634832859039307
-------epoch  336 -------
epoch: 336, train_loss: -5.592214107513428, valid_loss: -3.1563560962677
-------epoch  337 -------
epoch: 337, train_loss: -5.579751014709473, valid_loss: -3.268387794494629
-------epoch  338 -------
epoch: 338, train_loss: -5.55112886428833, valid_loss: -3.0717

...

-------epoch  1706 -------
epoch: 1706, train_loss: -8.622049331665039, valid_loss: -4.034895896911621
-------epoch  1707 -------
epoch: 1707, train_loss: -8.585198402404785, valid_loss: -4.108928680419922
-------epoch  1708 -------
epoch: 1708, train_loss: -8.629131317138672, valid_loss: -4.063742637634277
-------epoch  1709 -------
epoch: 1709, train_loss: -8.661632537841797, valid_loss: -4.116198539733887
-------epoch  1710 -------
epoch: 1710, train_loss: -8.712355613708496, valid_loss: -4.092911243438721
-------epoch  1711 -------
epoch: 1711, train_loss: -8.742587089538574, valid_loss: -4.113944053649902
-------epoch  1712 -------
epoch: 1712, train_loss: -8.761910438537598, valid_loss: -4.113283634185791
-------epoch  1713 -------
epoch: 1713, train_loss: -8.768271446228027, valid_loss: -4.092006683349609
-------epoch  1714 -------
epoch: 1714, train_loss: -8.763627052307129, valid_loss: -4.126086711883545
-------epoch  1715 -------
epoch: 1715, train_loss: -8.751001358032227, v

-------epoch  1786 -------
epoch: 1786, train_loss: -8.780351638793945, valid_loss: -4.026236057281494
-------epoch  1787 -------
epoch: 1787, train_loss: -8.776975631713867, valid_loss: -4.03023099899292
-------epoch  1788 -------
epoch: 1788, train_loss: -8.775591850280762, valid_loss: -4.019068717956543
-------epoch  1789 -------
epoch: 1789, train_loss: -8.775611877441406, valid_loss: -4.022008895874023
-------epoch  1790 -------
epoch: 1790, train_loss: -8.777235984802246, valid_loss: -4.010639667510986
-------epoch  1791 -------
epoch: 1791, train_loss: -8.779045104980469, valid_loss: -4.021975517272949
-------epoch  1792 -------
epoch: 1792, train_loss: -8.780990600585938, valid_loss: -4.0017476081848145
-------epoch  1793 -------
epoch: 1793, train_loss: -8.782012939453125, valid_loss: -4.017903804779053
-------epoch  1794 -------
epoch: 1794, train_loss: -8.782206535339355, valid_loss: -3.997431993484497
-------epoch  1795 -------
epoch: 1795, train_loss: -8.780293464660645, v

-------epoch  1866 -------
epoch: 1866, train_loss: -8.644050598144531, valid_loss: -3.9316961765289307
-------epoch  1867 -------
epoch: 1867, train_loss: -8.673039436340332, valid_loss: -3.902794599533081
-------epoch  1868 -------
epoch: 1868, train_loss: -8.691627502441406, valid_loss: -3.9430055618286133
-------epoch  1869 -------
epoch: 1869, train_loss: -8.72351360321045, valid_loss: -3.9308624267578125
-------epoch  1870 -------
epoch: 1870, train_loss: -8.742186546325684, valid_loss: -3.948837995529175
-------epoch  1871 -------
epoch: 1871, train_loss: -8.753847122192383, valid_loss: -3.9601783752441406
-------epoch  1872 -------
epoch: 1872, train_loss: -8.750387191772461, valid_loss: -3.9387054443359375
-------epoch  1873 -------
epoch: 1873, train_loss: -8.737811088562012, valid_loss: -3.9773309230804443
-------epoch  1874 -------
epoch: 1874, train_loss: -8.721409797668457, valid_loss: -3.9314773082733154
-------epoch  1875 -------
epoch: 1875, train_loss: -8.712528228759

-------epoch  1946 -------
epoch: 1946, train_loss: -8.768383026123047, valid_loss: -3.808587074279785
-------epoch  1947 -------
epoch: 1947, train_loss: -8.771875381469727, valid_loss: -3.8298380374908447
-------epoch  1948 -------
epoch: 1948, train_loss: -8.778352737426758, valid_loss: -3.799001932144165
-------epoch  1949 -------
epoch: 1949, train_loss: -8.781380653381348, valid_loss: -3.828031301498413
-------epoch  1950 -------
epoch: 1950, train_loss: -8.786065101623535, valid_loss: -3.7919092178344727
-------epoch  1951 -------
epoch: 1951, train_loss: -8.787446022033691, valid_loss: -3.82025408744812
-------epoch  1952 -------
epoch: 1952, train_loss: -8.790510177612305, valid_loss: -3.7851803302764893
-------epoch  1953 -------
epoch: 1953, train_loss: -8.7909574508667, valid_loss: -3.810955762863159
-------epoch  1954 -------
epoch: 1954, train_loss: -8.79295539855957, valid_loss: -3.775780439376831
-------epoch  1955 -------
epoch: 1955, train_loss: -8.792686462402344, va

-------epoch  2028 -------
epoch: 2028, train_loss: -8.759842872619629, valid_loss: inf
-------epoch  2029 -------
epoch: 2029, train_loss: -8.722448348999023, valid_loss: inf
-------epoch  2030 -------
epoch: 2030, train_loss: -8.666813850402832, valid_loss: inf
-------epoch  2031 -------
epoch: 2031, train_loss: -8.676840782165527, valid_loss: inf
-------epoch  2032 -------
epoch: 2032, train_loss: -8.678327560424805, valid_loss: inf
-------epoch  2033 -------
epoch: 2033, train_loss: -8.72639274597168, valid_loss: inf
-------epoch  2034 -------
epoch: 2034, train_loss: -8.745866775512695, valid_loss: inf
early stop at: 2034; optimal epoch at: 1233


In [45]:
starting_epoch = 30
end_epoch = 1529
train_loss_curve, = plt.plot(list(range(starting_epoch, end_epoch)), train_loss[starting_epoch:end_epoch])
valid_loss_curve, = plt.plot(list(range(starting_epoch, end_epoch)), valid_loss[starting_epoch:end_epoch])
plt.legend([train_loss_curve, valid_loss_curve], ['Train Loss', 'Validation Loss'])
plt.grid()
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")

Text(0, 0.5, 'MSE Loss')

In [ ]:
epoch: 728, train_loss: -7.462584972381592, valid_loss: -4.700007915496826



In [46]:
load_model_mdn = MDN(2, n_hidden=20, n_gaussians=num_gaussians)
load_model_mdn.load_state_dict(torch.load(PATH))
load_model_mdn.eval()
# mmodel = torch.load(PATH)

MDN(
  (l_h): Sequential(
    (0): Linear(in_features=2, out_features=20, bias=True)
    (1): Tanh()
  )
  (l_pi): Linear(in_features=20, out_features=6, bias=True)
  (l_mu_theta): Linear(in_features=20, out_features=6, bias=True)
  (l_sigma_theta): Linear(in_features=20, out_features=6, bias=True)
  (l_mu_phi): Linear(in_features=20, out_features=6, bias=True)
  (l_sigma_phi): Linear(in_features=20, out_features=6, bias=True)
  (l_correlation_theta_phi): Sequential(
    (0): Linear(in_features=20, out_features=6, bias=True)
    (1): Tanh()
  )
)

In [47]:
# Calculating validation data loss
pi_valid, mu_theta_valid, mu_phi_valid, sigma_theta_valid, sigma_phi_valid, correlation_theta_phi_valid = load_model_mdn(pd_thetaphi_valid_label_tensor)
loss_valid = mdn_loss_fn(gt_thetaphi_valid_label_tensor,
                         pi_valid, mu_theta_valid,
                         mu_phi_valid,
                         sigma_theta_valid,
                         sigma_phi_valid,
                         correlation_theta_phi_valid)
print(loss_valid.data.item())

-4.700007915496826


In [48]:
np.save('4000/without_normalization_with_earlystop/ng_6/train_loss.npy', np.asarray(train_loss))
np.save('4000/without_normalization_with_earlystop/ng_6/valid_loss.npy', np.asarray(valid_loss))

In [ ]:
a = torch.rand(2, 3, 4)

In [ ]:
a

In [ ]:
F.softmax(a, -1)

In [ ]:
a.repeat(1, 1, 1).shape

In [ ]:
epoch

In [ ]:
pi, mu_theta, mu_phi, sigma_theta, sigma_phi, correlation_theta_phi = model_mdn(pd_thetaphi_label_tensor)

size = pi.shape[0]
n_gaussians = pi.shape[1]
# print("sample size: ", size)
# print("num of gaus: ", n_gaussians)
# print("correlation size: ", correlation_x_y.shape)
# build mean matrix
mean = torch.stack((mu_theta, mu_phi), dim=2)
# build covariance matrix with standard bivariate normal distribution
cov = torch.zeros([size, n_gaussians, 2, 2])
cov[:, :, 0, 0] = sigma_theta**2
cov[:, :, 1, 1] = sigma_phi**2
cov[:, :, 1, 0] = correlation_theta_phi*sigma_theta*sigma_phi
cov[:, :, 0, 1] = correlation_theta_phi*sigma_theta*sigma_phi

m = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)


In [ ]:
pi.shape

In [ ]:
pd = m.rsample()

In [ ]:
k = torch.multinomial(pi, 1).view(-1)
PPD = torch.zeros([size, 2])
for i in range(pd.shape[0]):
    PPD[i, :] = pd[i,k[i],:]

In [ ]:
PPD.shape

In [ ]:
PPD = PPD.detach().numpy()

In [ ]:
mu_theta = mu_theta.detach().numpy()
mu_phi = mu_phi.detach().numpy()

In [ ]:
gt_thetaphi_label = gt_thetaphi_label_tensor.numpy()
pd_thetaphi_label = pd_thetaphi_label_tensor.numpy()

In [ ]:
type(gt_thetaphi_label)

In [ ]:
correlation_theta_phi

In [ ]:
plt.plot(gt_thetaphi_label[:,0], label='gt', linestyle='--')
plt.plot(pd_thetaphi_label[:,0], label='pd', linestyle='--')
plt.plot(PPD[:,0], label='pd_mdn', linestyle='--')
plt.plot(mu_theta[:,0], label='mu 1')
plt.plot(mu_theta[:,1], label='mu 2')
# plt.plot(mu_x[:,2], label='mu 3')
# plt.plot(mu_x[:,3], label='mu 4')
# plt.plot(mu_x[:,4], label='mu 5')
plt.legend()

In [ ]:
plt.plot(gt_thetaphi_label[:,1], label='gt', linestyle='--')
plt.plot(pd_thetaphi_label[:,1], label='pd', linestyle='--')
plt.plot(PPD[:,1], label='pd_mdn', linestyle='--')
plt.plot(mu_phi[:,0], label='mu 1')
plt.plot(mu_phi[:,1], label='mu 2')
# plt.plot(mu_x[:,2], label='mu 3')
# plt.plot(mu_x[:,3], label='mu 4')
# plt.plot(mu_x[:,4], label='mu 5')
plt.legend()

In [ ]:
type(mu_theta)

In [ ]:
mu_theta = mu_theta*180
mu_phi = mu_phi*360

In [ ]:
sigma_theta = sigma_theta*180
sigma_phi = sigma_phi*360

In [ ]:
mu_theta

In [ ]:
mu_phi

In [ ]:
pi

In [ ]:
gt_label
pd_label

In [ ]:
gt_label.shape

In [ ]:
def getXYZ(theta, phi, d):
    cos_theta = math.cos(theta * math.pi / 180.0 );
    sin_theta = math.sin(theta * math.pi / 180.0 );
    cos_phi   = math.cos(phi   * math.pi / 180.0 );
    sin_phi   = math.sin(phi   * math.pi / 180.0 );

    x = d * sin_theta * cos_phi;
    y = d * sin_theta * sin_phi;
    z = d * cos_theta;
    return x, y, z

In [ ]:
def getCircle(theta_list, phi_list, d):
    size = len(theta_list)
    x_list = []
    y_list = []
    z_list = []
    for i in range(size):
        x, y, z = getXYZ(theta_list[i], phi_list[i], d)
        # print(math.sqrt(x**2 + y**2 + z**2))
        x_list.append(x)
        y_list.append(y)
        z_list.append(z)
        
        
    return x_list, y_list, z_list

In [ ]:
def getPhi(theta, sigma_theta, sigma_phi):
    return math.sqrt((1-theta*theta/sigma_theta/sigma_theta)*sigma_phi*sigma_phi)

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
# add origin
ax.scatter3D([0], [0], [0], cmap='Red');

In [ ]:
cam_x = []
cam_y = []
cam_z = []

for i in range(10):
    index = i
    selected_mean_theta = mu_theta[index, 1]
    selected_sigma_theta = sigma_theta[index, 1] * 3
    selected_mean_phi = mu_phi[index, 1]
    selected_sigma_phi = sigma_phi[index, 1] * 3

    half_phi = getPhi(selected_sigma_theta/2, selected_sigma_theta, selected_sigma_phi)

    theta_list = [selected_mean_theta + selected_sigma_theta,
                  selected_mean_theta + selected_sigma_theta/2,
                  selected_mean_theta,
                  selected_mean_theta - selected_sigma_theta/2,
                  selected_mean_theta - selected_sigma_theta,
                  selected_mean_theta - selected_sigma_theta/2,
                  selected_mean_theta,
                  selected_mean_theta + selected_sigma_theta/2,
                  selected_mean_theta + selected_sigma_theta]
    phi_list   = [selected_mean_phi,
                  selected_mean_phi - half_phi,
                  selected_mean_phi - selected_sigma_phi,
                  selected_mean_phi - half_phi,
                  selected_mean_phi,
                  selected_mean_phi + half_phi,
                  selected_mean_phi + selected_sigma_phi,
                  selected_mean_phi + half_phi,
                  selected_mean_phi]

    x = pd_label[index, 0]
    y = pd_label[index, 1]
    z = pd_label[index, 2]
    
    cam_x.append(x)
    cam_y.append(y)
    cam_z.append(z)

    d = math.sqrt(x**2 + y**2 + z**2)
    # print("d", d)

    x_list, y_list, z_list = getCircle(theta_list, phi_list, d)

    ax.plot3D(x_list, y_list, z_list, 'red')
    # ax.scatter3D(x_list, y_list, z_list, 'gray')
    # ax.scatter3D(x, y, z, cmap='Red');

In [ ]:
ax.plot3D(cam_x, cam_y, cam_z, 'gray')

In [ ]:
index = 0
selected_mean_theta = mu_theta[index, 1]
selected_sigma_theta = sigma_theta[index, 1] * 3
selected_mean_phi = mu_phi[index, 1]
selected_sigma_phi = sigma_phi[index, 1] * 3

half_phi = getPhi(selected_sigma_theta/2, selected_sigma_theta, selected_sigma_phi)

theta_list = [selected_mean_theta + selected_sigma_theta,
              selected_mean_theta + selected_sigma_theta/2,
              selected_mean_theta,
              selected_mean_theta - selected_sigma_theta/2,
              selected_mean_theta - selected_sigma_theta,
              selected_mean_theta - selected_sigma_theta/2,
              selected_mean_theta,
              selected_mean_theta + selected_sigma_theta/2,
              selected_mean_theta + selected_sigma_theta]
phi_list   = [selected_mean_phi,
              selected_mean_phi - half_phi,
              selected_mean_phi - selected_sigma_phi,
              selected_mean_phi - half_phi,
              selected_mean_phi,
              selected_mean_phi + half_phi,
              selected_mean_phi + selected_sigma_phi,
              selected_mean_phi + half_phi,
              selected_mean_phi]

x = pd_label[index, 0]
y = pd_label[index, 1]
z = pd_label[index, 2]

d = math.sqrt(x**2 + y**2 + z**2)
print("d", d)

x_list, y_list, z_list = getCircle(theta_list, phi_list, d)

ax.plot3D(x_list, y_list, z_list, 'gray')
# ax.scatter3D(x_list, y_list, z_list, 'gray')
ax.scatter3D(x, y, z, cmap='Red');

In [ ]:
index = 1
selected_mean_theta = mu_theta[index, 1]
selected_sigma_theta = sigma_theta[index, 1] * 3
selected_mean_phi = mu_phi[index, 1]
selected_sigma_phi = sigma_phi[index, 1] * 3

half_phi = getPhi(selected_sigma_theta/2, selected_sigma_theta, selected_sigma_phi)

theta_list = [selected_mean_theta + selected_sigma_theta,
              selected_mean_theta + selected_sigma_theta/2,
              selected_mean_theta,
              selected_mean_theta - selected_sigma_theta/2,
              selected_mean_theta - selected_sigma_theta,
              selected_mean_theta - selected_sigma_theta/2,
              selected_mean_theta,
              selected_mean_theta + selected_sigma_theta/2,
              selected_mean_theta + selected_sigma_theta]
phi_list   = [selected_mean_phi,
              selected_mean_phi - half_phi,
              selected_mean_phi - selected_sigma_phi,
              selected_mean_phi - half_phi,
              selected_mean_phi,
              selected_mean_phi + half_phi,
              selected_mean_phi + selected_sigma_phi,
              selected_mean_phi + half_phi,
              selected_mean_phi]

x = pd_label[index, 0]
y = pd_label[index, 1]
z = pd_label[index, 2]

d = math.sqrt(x**2 + y**2 + z**2)
print("d", d)

x_list, y_list, z_list = getCircle(theta_list, phi_list, d)

ax.plot3D(x_list, y_list, z_list, 'gray')
# ax.scatter3D(x_list, y_list, z_list, 'gray')
ax.scatter3D(x, y, z, cmap='Red');

In [ ]:
d = math.sqrt(gt_label[0, 0]**2 + gt_label[0, 0]**2 + gt_label[0, 0]**2)

In [ ]:
index = 1
x = pd_label[index, 0]
y = pd_label[index, 1]
z = pd_label[index, 2]

d = math.sqrt(x**2 + y**2 + z**2)
print("d", d)

x_list, y_list, z_list = getCircle(theta_list, phi_list, d)


In [ ]:
ax.plot3D(x_list, y_list, z_list, 'gray')
# ax.scatter3D(x_list, y_list, z_list, 'gray')
ax.scatter3D(x, y, z, cmap='Red');

In [ ]:
index = 0
radius = 0.2
density = 40 # sample size alone radius
x_center, y_center, z_center = pd_label[index]

s_x = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
s_y = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
s_z = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])

x_min = x_center - radius
y_min = y_center - radius
z_min = z_center - radius
unit_size = radius/density

for i in range(density*2 + 1):
    for j in range(density*2 + 1):
        for k in range(density*2 + 1):
            s_x[i, j, k] = x_min + i*unit_size
            s_y[i, j, k] = y_min + j*unit_size
            s_z[i, j, k] = z_min + k*unit_size
            


pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z = model_mdn(pd_label_tensor[index])
pi = pi.unsqueeze(0)
mu_x = mu_x.unsqueeze(0)
mu_y = mu_y.unsqueeze(0)
mu_z = mu_z.unsqueeze(0)
sigma_x = sigma_x.unsqueeze(0)
sigma_y = sigma_y.unsqueeze(0)
sigma_z = sigma_z.unsqueeze(0)
# print(pi.shape)
# print(mu_x.shape)
# print(mu_y.shape)
# print(mu_z.shape)
# print(sigma_x.shape)
# print(sigma_y.shape)
# print(sigma_z.shape)

size = pi.shape[0]
n_gaussians = pi.shape[1]
# build mean matrix
mean = torch.stack((mu_x, mu_y, mu_z), dim=2)
# print(mean.shape)
# build covariance matrix with standard trivariate normal distribution
cov = torch.zeros([size, n_gaussians, 3, 3])
cov[:, :, 0, 0] = sigma_x**2
cov[:, :, 1, 1] = sigma_y**2
cov[:, :, 2, 2] = sigma_z**2
# print(cov.shape)

# mean[0, 0] = -0.18
# mean[0, 1] = 0
# mean[0, 2] = -0.01

normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)

pdf = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
for i in range(density*2 + 1):
    print(i)
    for j in range(density*2 + 1):
        for k in range(density*2 + 1):
            likelihood = torch.exp(normal_3d.log_prob(torch.tensor([[s_x[i, j, k], s_y[i, j, k], s_z[i, j, k]]])))
            # get mixture sum
            # pdf[i, j, k] = torch.sum(likelihood * pi, dim=1)
            pdf[i, j, k] = likelihood[0, 0]
        


In [ ]:
index = 0
radius = 0.2
density = 40 # sample size alone radius
x_center, y_center, z_center = pd_label[index]

s_x = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
s_y = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
s_z = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])

x_min = x_center - radius
y_min = y_center - radius
z_min = z_center - radius
unit_size = radius/density

for i in range(density*2 + 1):
    for j in range(density*2 + 1):
        for k in range(density*2 + 1):
            s_x[i, j, k] = x_min + i*unit_size
            s_y[i, j, k] = y_min + j*unit_size
            s_z[i, j, k] = z_min + k*unit_size
            


pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z = model_mdn(pd_label_tensor[index])
pi = pi.unsqueeze(0)
mu_x = mu_x.unsqueeze(0)
mu_y = mu_y.unsqueeze(0)
mu_z = mu_z.unsqueeze(0)
sigma_x = sigma_x.unsqueeze(0)
sigma_y = sigma_y.unsqueeze(0)
sigma_z = sigma_z.unsqueeze(0)
# print(pi.shape)
# print(mu_x.shape)
# print(mu_y.shape)
# print(mu_z.shape)
# print(sigma_x.shape)
# print(sigma_y.shape)
# print(sigma_z.shape)

size = pi.shape[0]
n_gaussians = pi.shape[1]
# build mean matrix
mean = torch.stack((mu_x, mu_y, mu_z), dim=2)
# print(mean.shape)
# build covariance matrix with standard trivariate normal distribution
cov = torch.zeros([size, n_gaussians, 3, 3])
cov[:, :, 0, 0] = sigma_x**2
cov[:, :, 1, 1] = sigma_y**2
cov[:, :, 2, 2] = sigma_z**2
# print(cov.shape)

# mean[0, 0] = 0.01
# mean[0, 1] = 0
# mean[0, 2] = -0.01

normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)

pdf_1 = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
for i in range(density*2 + 1):
    print(i)
    for j in range(density*2 + 1):
        for k in range(density*2 + 1):
            likelihood = torch.exp(normal_3d.log_prob(torch.tensor([[s_x[i, j, k], s_y[i, j, k], s_z[i, j, k]]])))
            # get mixture sum
            # pdf_![i, j, k] = torch.sum(likelihood * pi, dim=1)
            pdf_1[i, j, k] = likelihood[0, 1]



In [ ]:
index = 0
radius = 0.2
density = 40 # sample size alone radius
# radius = 0.01
# density = 20 # sample size alone radius
x_center, y_center, z_center = pd_label[index]

s_x = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
s_y = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
s_z = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])

x_min = x_center - radius
y_min = y_center - radius
z_min = z_center - radius
unit_size = radius/density

for i in range(density*2 + 1):
    for j in range(density*2 + 1):
        for k in range(density*2 + 1):
            s_x[i, j, k] = x_min + i*unit_size
            s_y[i, j, k] = y_min + j*unit_size
            s_z[i, j, k] = z_min + k*unit_size
            


pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z = model_mdn(pd_label_tensor[index])
pi = pi.unsqueeze(0)
mu_x = mu_x.unsqueeze(0)
mu_y = mu_y.unsqueeze(0)
mu_z = mu_z.unsqueeze(0)
sigma_x = sigma_x.unsqueeze(0)
sigma_y = sigma_y.unsqueeze(0)
sigma_z = sigma_z.unsqueeze(0)
# print(pi.shape)
# print(mu_x.shape)
# print(mu_y.shape)
# print(mu_z.shape)
# print(sigma_x.shape)
# print(sigma_y.shape)dd
# print(sigma_z.shape)

size = pi.shape[0]
n_gaussians = pi.shape[1]
# build mean matrix
mean = torch.stack((mu_x, mu_y, mu_z), dim=2)
# print(mean.shape)
# build covariance matrix with standard trivariate normal distribution
cov = torch.zeros([size, n_gaussians, 3, 3])
cov[:, :, 0, 0] = sigma_x**2
cov[:, :, 1, 1] = sigma_y**2
cov[:, :, 2, 2] = sigma_z**2
# print(cov.shape)

# mean[0, 0] = -0.18
# mean[0, 1] = 0
# mean[0, 2] = -0.01

normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)

        
pdf_all = np.zeros([density*2 + 1, density*2 + 1, density*2 + 1])
for i in range(density*2 + 1):
    print(i)
    for j in range(density*2 + 1):
        for k in range(density*2 + 1):
            likelihood = torch.exp(normal_3d.log_prob(torch.tensor([[s_x[i, j, k], s_y[i, j, k], s_z[i, j, k]]])))
            # get mixture sum
            pdf_all[i, j, k] = torch.sum(likelihood * pi, dim=1)
            # pdf_4[i, j, k] = likelihood[0, 4]



In [ ]:
print(pd_label_tensor[0])
print(pi[0])
print(mu_x[0])
print(mu_y[0])
print(mu_z[0])
print(sigma_x[0])
print(sigma_y[0])
print(sigma_z[0])

In [ ]:
import numpy as np
from mayavi import mlab

src = mlab.pipeline.scalar_field(pdf)
mlab.pipeline.iso_surface(src, contours=[0.1, ], opacity=0.3, color = (0.8,0.0,0.0))
# mlab.pipeline.iso_surface(src, contours=[pdf.max()-0.1*pdf.ptp(), ],)

src_1 = mlab.pipeline.scalar_field(pdf_1)
mlab.pipeline.iso_surface(src_1, contours=[0.1, ], opacity=0.3, color = (0.8,0.0,0.0))
# mlab.pipeline.iso_surface(src_1, contours=[pdf_1.max()-0.1*pdf_1.ptp(), ],)

# src_half = mlab.pipeline.scalar_field(pdf_half)
# mlab.pipeline.iso_surface(src_half, contours=[0.1, ], opacity=0.3, color = (0.0,0.8,0.0))
# mlab.pipeline.iso_surface(src, contours=[pdf.max()-0.1*pdf.ptp(), ],)

# src_1_half = mlab.pipeline.scalar_field(pdf_1_half)
# mlab.pipeline.iso_surface(src_1_half, contours=[0.1, ], opacity=0.3, color = (0.0,0.8,0.0))
# mlab.pipeline.iso_surface(src_1, contours=[pdf_1.max()-0.1*pdf_1.ptp(), ],)

src_all = mlab.pipeline.scalar_field(pdf_all)
mlab.pipeline.iso_surface(src_all, contours=[0.1, ], opacity=0.3, color = (0.0,0.0,0.8))
# mlab.pipeline.iso_surface(src_1, contours=[pdf_1.max()-0.1*pdf_1.ptp(), ],)

In [ ]:
8.1318e-02 + 3.6224e-01 + 9.2028e-07 + 1.2253e-02 + 1.7447e-06

In [ ]:
pi

In [ ]:
sigma_x

In [ ]:
sigma_y

In [ ]:
sigma_z

In [ ]:
mu_x

In [ ]:
pi.unsqueeze(0).shape

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt



fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')




ax.scatter(s_x.flatten(), s_y.flatten(), s_z.flatten(), c='r', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
pd_label[0]

In [ ]:
a

In [ ]:
b

In [ ]:
c

In [ ]:
pd_label[0]

In [ ]:

for i in range(21):
    for j in range(21):
        for k in range(21):
            s[i, j, k] = torch.exp(normal_3d.log_prob(torch.FloatTensor([i-10, j-10, k-10])))
            
src = mlab.pipeline.scalar_field(s)
mlab.pipeline.iso_surface(src, contours=[s.min()+0.1*s.ptp(), ], opacity=0.3)
mlab.pipeline.iso_surface(src, contours=[s.max()-0.1*s.ptp(), ],)

In [ ]:
    m = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)

k = torch.multinomial(pi, 1).view(-1)
y_pred = torch.normal(mu, sigma)[np.arange(n_samples), k].data

In [ ]:
a = torch.tensor([0.1585, 0.1787, 0.1571, 0.3795, 0.1261])

In [ ]:
a = torch.tensor([0.0165, 0.0245, 0.1137, 2.8152, 0.0082])
b = torch.tensor([0.0339, 0.0643, 0.1532, 0.5262, 0.2225])

In [ ]:
torch.sum(a*b, dim=0)

In [ ]:
(a*b).shape

In [ ]:
a = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12]])

In [ ]:

a

In [ ]:
a[:, [1,2]]

In [ ]:
a.shape

In [ ]:
c

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
c.shape

In [ ]:
d = torch.stack((a, b), dim=1)

In [ ]:
d

In [ ]:
a.dtype

In [ ]:
d[:,:,0]

In [ ]:
d[0,0,:]

In [ ]:
F.softmax(a, -1)

In [ ]:
a

In [ ]:
math.exp(7) / (math.exp(1) + math.exp(7))

In [ ]:
F.softmax(a, 1)

In [ ]:
a.shape

In [ ]:
mean = torch.stack((mu_x, mu_y, mu_z), dim=2)

In [ ]:
w[:, :, 0, 0].shape

In [ ]:
n_gaussians = 5
size = 14000

In [ ]:
e = e.unsqueeze(0).repeat(n_gaussians, 1, 1).unsqueeze(0).repeat(size, 1, 1, 1)

In [ ]:
e.shape

In [ ]:
e = e.repeat(5, 1, 1)

In [ ]:
e.shape

In [ ]:
e = e.unsqueeze(0)

In [ ]:
e.shape

In [ ]:
e = e.repeat(10, 1, 1, 1)

In [ ]:
e.shape

In [ ]:
e[0][0]

In [ ]:
a = torch.tensor([[1,2,3],[4,5,6]])

In [ ]:
a

In [ ]:
a**2

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
# ax.plot3D(test[:,0], test[:,1], test[:,2], 'gray')
ax.plot3D(input_tensor[0, :, 0], input_tensor[0, :, 1], input_tensor[0, :, 2], 'gray')
ax.scatter(gt_label_tensor[0,0], gt_label_tensor[0,1], gt_label_tensor[0,2], 'green')
ax.scatter(pd_label_tensor[0,0], pd_label_tensor[0,1], pd_label_tensor[0,2], 'red')
# ax.plot3D(gt_label_tensor[0:2,0], gt_label_tensor[0:2,1], gt_label_tensor[0:2,2], 'green')
# ax.plot3D(pd_label_tensor[0:2,0], pd_label_tensor[0:2,1], pd_label_tensor[0:2,2], 'red')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
# ax.plot3D(test[:,0], test[:,1], test[:,2], 'gray')
ax.plot3D(input_tensor[:3, 0, 0], input_tensor[:3, 0, 1], input_tensor[:3, 0, 2], 'gray')
ax.scatter(input_tensor[:3, 0, 0], input_tensor[:3, 0, 1], input_tensor[:3, 0, 2], c = 'gray')

ax.plot3D(input_tensor[2:6, 0, 0], input_tensor[2:6, 0, 1], input_tensor[2:6, 0, 2], 'green')
ax.scatter(input_tensor[2:6, 0, 0], input_tensor[2:6, 0, 1], input_tensor[2:6, 0, 2], c = 'green')

ax.plot3D(gt_label_tensor[0:3,0], gt_label_tensor[0:3,1], gt_label_tensor[0:3,2], 'black')
ax.plot3D(pd_label_tensor[0:3,0], pd_label_tensor[0:3,1], pd_label_tensor[0:3,2], 'red')
ax.scatter(pd_label_tensor[0:3,0], pd_label_tensor[0:3,1], pd_label_tensor[0:3,2], c = 'red')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

In [ ]:
x_pd = pd_label_tensor[:, 0].reshape(-1,1)
y_pd = pd_label_tensor[:, 1].reshape(-1,1)
z_pd = pd_label_tensor[:, 2].reshape(-1,1)

x_gt = gt_label_tensor[:, 0].reshape(-1,1)
y_gt = gt_label_tensor[:, 1].reshape(-1,1)
z_gt = gt_label_tensor[:, 2].reshape(-1,1)

In [ ]:
x_gt = x_gt.reshape(-1,1)

In [ ]:
x_gt.shape

In [ ]:
plt.plot(x_gt)
plt.plot(x_pd)

### Apply MDN on x

In [ ]:
n_samples = 14985

x_data = torch.Tensor(x_pd)
y_data = torch.Tensor(x_gt)

plt.figure(figsize=(8, 8))
plt.scatter(x_data, y_data, alpha=0.4)
plt.show()

In [ ]:
x_data.shape

In [ ]:
class MDN(nn.Module):
    def __init__(self, n_hidden, n_gaussians):
        super(MDN, self).__init__()
        self.z_h = nn.Sequential(
            nn.Linear(1, n_hidden),
            nn.Tanh()
        )
        self.z_pi = nn.Linear(n_hidden, n_gaussians)
        self.z_mu = nn.Linear(n_hidden, n_gaussians)
        self.z_sigma = nn.Linear(n_hidden, n_gaussians)
    
    def forward(self, x):
        z_h = self.z_h(x)
        pi = F.softmax(self.z_pi(z_h), -1)
        mu = self.z_mu(z_h)
        sigma = torch.exp(self.z_sigma(z_h))
        return pi, mu, sigma

model = MDN(n_hidden=20, n_gaussians=5)

optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def mdn_loss_fn(y, mu, sigma, pi):
    m = torch.distributions.Normal(loc=mu, scale=sigma)
    loss = torch.exp(m.log_prob(y))
    loss = torch.sum(loss * pi, dim=1)
    loss = -torch.log(loss)
    return torch.mean(loss)

In [ ]:
for epoch in range(10000):
    pi, mu, sigma = model(x_data)
    loss = mdn_loss_fn(y_data, mu, sigma, pi)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(loss.data.tolist())

In [ ]:
pi, mu, sigma = model(x_data)

In [ ]:
print(pi.shape)
print(mu.shape)
print(sigma.shape)

In [ ]:
# get the index of the highest index of the 5 elements in each row
k = torch.multinomial(pi, 1).view(-1)
y_pred = torch.normal(mu, sigma)[np.arange(n_samples), k].data

In [ ]:
plt.figure(figsize=(8, 8))
# lt.scatter(x_data, y_data, alpha=0.4)
# plt.scatter(x_data, y_pred, alpha=0.8, color='red')
# plt.scatter(x_test[0], y_pred[0], alpha=0.4, color='yellow')
plt.scatter(x_data, mu[:,0].detach(), label='mean 0')
plt.scatter(x_data, mu[:,1].detach(), label='mean 1')
plt.scatter(x_data, mu[:,2].detach(), label='mean 2')
plt.scatter(x_data, mu[:,3].detach(), label='mean 3')
plt.scatter(x_data, mu[:,4].detach(), label='mean 4')
# plt.plot(x_test.numpy(), mu[:,0].detach().numpy())
# plt.scatter(x_data[:20], y_data[:20], alpha=0.4)
plt.legend()
plt.show()

In [ ]:
pi = pi.detach().numpy()
mu = mu.detach().numpy()
sigma = sigma.detach().numpy()


In [ ]:
x_data = x_data.numpy()

In [ ]:
p = np.zeros((n_samples, 100))
x = np.linspace(-2, 2, 100)

for i in range(n_samples):
    each = []
    for j in range(5):
        if (j == 0):
            y = [pi[i, j] * ele for ele in scipy.stats.norm.pdf(x, mu[i, j],sigma[i, j])]
            each.append(y)
            cumulated_y = y
        else:
            y = [pi[i, j] * ele for ele in scipy.stats.norm.pdf(x, mu[i, j],sigma[i, j])]
            each.append(y)
            cumulated_y = [p + q for p, q in zip(cumulated_y, y)]
    p[i, :] = cumulated_y

In [ ]:
p.shape

In [ ]:
ax = plt.axes(projection='3d')
for i in range(0,n_samples):
    ax.plot3D([x_data[i, 0]]*100, x, p[i, :])

ax.set_xlabel('index')
ax.set_ylabel('x')
ax.set_zlabel('pdf')

In [ ]:
x_data[0, 1]

### apply MDN on 3D projection

In [ ]:
mean_1_1 = torch.Tensor([1, 1, 1])
mean_1_2 = torch.Tensor([2, 2, 2])
mean_1_3 = torch.Tensor([3, 3, 3])
mean_1_4 = torch.Tensor([4, 4, 4])
mean_1_5 = torch.Tensor([5, 5, 5])

mean_2_1 = torch.Tensor([6, 6, 6])
mean_2_2 = torch.Tensor([7, 7, 7])
mean_2_3 = torch.Tensor([8, 8, 8])
mean_2_4 = torch.Tensor([9, 9, 9])
mean_2_5 = torch.Tensor([10, 10, 10])

sigma_1 = 1
sigma_2 = 1
sigma_3 = 1
cov_1_1 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_1_1[1, 0] = rho_1_2*sigma_1*sigma_2
cov_1_1[2, 0] = rho_1_3*sigma_1*sigma_3
cov_1_1[2, 1] = rho_2_3*sigma_2*sigma_3
cov_1_1[0, 1] = rho_1_2*sigma_1*sigma_2
cov_1_1[0, 2] = rho_1_3*sigma_1*sigma_3
cov_1_1[1, 2] = rho_2_3*sigma_2*sigma_3

sigma_1 = 2
sigma_2 = 2
sigma_3 = 2
cov_1_2 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_1_2[1, 0] = rho_1_2*sigma_1*sigma_2
cov_1_2[2, 0] = rho_1_3*sigma_1*sigma_3
cov_1_2[2, 1] = rho_2_3*sigma_2*sigma_3
cov_1_2[0, 1] = rho_1_2*sigma_1*sigma_2
cov_1_2[0, 2] = rho_1_3*sigma_1*sigma_3
cov_1_2[1, 2] = rho_2_3*sigma_2*sigma_3
                         
sigma_1 = 3
sigma_2 = 3
sigma_3 = 3
cov_1_3 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_1_3[1, 0] = rho_1_2*sigma_1*sigma_2
cov_1_3[2, 0] = rho_1_3*sigma_1*sigma_3
cov_1_3[2, 1] = rho_2_3*sigma_2*sigma_3
cov_1_3[0, 1] = rho_1_2*sigma_1*sigma_2
cov_1_3[0, 2] = rho_1_3*sigma_1*sigma_3
cov_1_3[1, 2] = rho_2_3*sigma_2*sigma_3
                         
sigma_1 = 4
sigma_2 = 4
sigma_3 = 4
cov_1_4 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_1_4[1, 0] = rho_1_2*sigma_1*sigma_2
cov_1_4[2, 0] = rho_1_3*sigma_1*sigma_3
cov_1_4[2, 1] = rho_2_3*sigma_2*sigma_3
cov_1_4[0, 1] = rho_1_2*sigma_1*sigma_2
cov_1_4[0, 2] = rho_1_3*sigma_1*sigma_3
cov_1_4[1, 2] = rho_2_3*sigma_2*sigma_3
                         
sigma_1 = 5
sigma_2 = 5
sigma_3 = 5
cov_1_5 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_1_5[1, 0] = rho_1_2*sigma_1*sigma_2
cov_1_5[2, 0] = rho_1_3*sigma_1*sigma_3
cov_1_5[2, 1] = rho_2_3*sigma_2*sigma_3
cov_1_5[0, 1] = rho_1_2*sigma_1*sigma_2
cov_1_5[0, 2] = rho_1_3*sigma_1*sigma_3
cov_1_5[1, 2] = rho_2_3*sigma_2*sigma_3
                         
sigma_1 = 6
sigma_2 = 6
sigma_3 = 6
cov_2_1 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_2_1[1, 0] = rho_1_2*sigma_1*sigma_2
cov_2_1[2, 0] = rho_1_3*sigma_1*sigma_3
cov_2_1[2, 1] = rho_2_3*sigma_2*sigma_3
cov_2_1[0, 1] = rho_1_2*sigma_1*sigma_2
cov_2_1[0, 2] = rho_1_3*sigma_1*sigma_3
cov_2_1[1, 2] = rho_2_3*sigma_2*sigma_3

sigma_1 = 7
sigma_2 = 7
sigma_3 = 7
cov_2_2 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_2_2[1, 0] = rho_1_2*sigma_1*sigma_2
cov_2_2[2, 0] = rho_1_3*sigma_1*sigma_3
cov_2_2[2, 1] = rho_2_3*sigma_2*sigma_3
cov_2_2[0, 1] = rho_1_2*sigma_1*sigma_2
cov_2_2[0, 2] = rho_1_3*sigma_1*sigma_3
cov_2_2[1, 2] = rho_2_3*sigma_2*sigma_3
                         
sigma_1 = 8
sigma_2 = 8
sigma_3 = 8
cov_2_3 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_2_3[1, 0] = rho_1_2*sigma_1*sigma_2
cov_2_3[2, 0] = rho_1_3*sigma_1*sigma_3
cov_2_3[2, 1] = rho_2_3*sigma_2*sigma_3
cov_2_3[0, 1] = rho_1_2*sigma_1*sigma_2
cov_2_3[0, 2] = rho_1_3*sigma_1*sigma_3
cov_2_3[1, 2] = rho_2_3*sigma_2*sigma_3
                         
sigma_1 = 9
sigma_2 = 9
sigma_3 = 9
cov_2_4 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_2_4[1, 0] = rho_1_2*sigma_1*sigma_2
cov_2_4[2, 0] = rho_1_3*sigma_1*sigma_3
cov_2_4[2, 1] = rho_2_3*sigma_2*sigma_3
cov_2_4[0, 1] = rho_1_2*sigma_1*sigma_2
cov_2_4[0, 2] = rho_1_3*sigma_1*sigma_3
cov_2_4[1, 2] = rho_2_3*sigma_2*sigma_3
                         
sigma_1 = 10
sigma_2 = 10
sigma_3 = 10
cov_2_5 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_2_5[1, 0] = rho_1_2*sigma_1*sigma_2
cov_2_5[2, 0] = rho_1_3*sigma_1*sigma_3
cov_2_5[2, 1] = rho_2_3*sigma_2*sigma_3
cov_2_5[0, 1] = rho_1_2*sigma_1*sigma_2
cov_2_5[0, 2] = rho_1_3*sigma_1*sigma_3
cov_2_5[1, 2] = rho_2_3*sigma_2*sigma_3

In [ ]:
mean_1 = torch.stack((mean_1_1, mean_1_2, mean_1_3, mean_1_4, mean_1_5), dim=0)

In [ ]:
mean_2 = torch.stack((mean_2_1, mean_2_2, mean_2_3, mean_2_4, mean_2_5), dim=0)

In [ ]:
mean = torch.stack((mean_1, mean_2), dim=0)

In [ ]:
cov_1 = torch.stack((cov_1_1, cov_1_2, cov_1_3, cov_1_4, cov_1_5), dim=0)
cov_2 = torch.stack((cov_2_1, cov_2_2, cov_2_3, cov_2_4, cov_2_5), dim=0)

In [ ]:
cov = torch.stack((cov_1, cov_2), dim=0)

In [ ]:
cov.shape

In [ ]:
mean.shape

In [ ]:
normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(mean, covariance_matrix=cov)

In [ ]:
y = torch.Tensor([[1,1,1], [2,2,2], [3,3,3], [4,4,4], [5,5,5]])

In [ ]:
y.shape

In [ ]:
y = torch.stack((y, y), dim=0)

In [ ]:
y.shape

In [ ]:
torch.exp(normal_3d.log_prob(y))

In [ ]:
r.shape

In [ ]:
r

In [ ]:
normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(mean_1, covariance_matrix=cov_1)

In [ ]:
y = torch.Tensor([[1,1,1], [2,2,2], [3,3,3], [4,4,4], [5,5,5]])

In [ ]:
y.shape

In [ ]:
mean_1 = mean_1.unsqueeze(0)
cov_1 = cov_1.unsqueeze(0)

In [ ]:
mean_1.shape

In [ ]:
cov_1.shape

In [ ]:
normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(mean_1, covariance_matrix=cov_1)

In [ ]:
y = torch.Tensor([[1,1,1], [2,2,2], [3,3,3], [4,4,4], [5,5,5]])

In [ ]:
y.shape

In [ ]:
y = y.unsqueeze(0)

In [ ]:
y.shape

In [ ]:
torch.exp(normal_3d.log_prob(y))

In [ ]:
y = torch.Tensor([[1,2,3]])

In [ ]:
y = torch.Tensor([[1, 1, 1],[1, 2, 3]])

In [ ]:
y = y.unsqueeze(1)

In [ ]:
y.shape

In [ ]:
y = y.unsqueeze(1)

In [ ]:
y.shape

In [ ]:
r = torch.exp(normal_3d.log_prob(y))

In [ ]:
r.shape

In [ ]:
r

In [ ]:
mean.shape

In [ ]:
cov = torch.stack((cov, cov_1), dim=0)

In [ ]:
cov.shape

In [ ]:
cov[0]

In [ ]:
cov[1]

In [ ]:
normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(mean, covariance_matrix=cov)

In [ ]:
y = torch.Tensor([[1, 1, 1],[1, 2, 3]])

In [ ]:
y = y.unsqueeze(1)

In [ ]:
y.shape

In [ ]:
torch.exp(normal_3d.log_prob(y))

In [ ]:
y = torch.Tensor([[1,1,1]])

In [ ]:
y.shape

In [ ]:
torch.exp(normal_3d.log_prob(y))

In [ ]:
y.shape

In [ ]:
mean = torch.zeros(3)
sigma_1 = 4
sigma_2 = 4
sigma_3 = 4
cov = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov[1, 0] = rho_1_2*sigma_1*sigma_2
cov[2, 0] = rho_1_3*sigma_1*sigma_3
cov[2, 1] = rho_2_3*sigma_2*sigma_3
cov[0, 1] = rho_1_2*sigma_1*sigma_2
cov[0, 2] = rho_1_3*sigma_1*sigma_3
cov[1, 2] = rho_2_3*sigma_2*sigma_3
normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(mean, covariance_matrix=cov)

y = torch.Tensor([[1, 2, 3]])
torch.exp(normal_3d.log_prob(y))

In [ ]:
mean = torch.zeros(3)
sigma_1 = 2
sigma_2 = 2
sigma_3 = 2
cov_1 = torch.Tensor([[sigma_1**2, 0, 0], [0, sigma_2**2, 0], [0, 0, sigma_3**2]])
rho_1_2 = 0.7
rho_2_3 = 0.7
rho_1_3 = 0.7
cov_1[1, 0] = rho_1_2*sigma_1*sigma_2
cov_1[2, 0] = rho_1_3*sigma_1*sigma_3
cov_1[2, 1] = rho_2_3*sigma_2*sigma_3
cov_1[0, 1] = rho_1_2*sigma_1*sigma_2
cov_1[0, 2] = rho_1_3*sigma_1*sigma_3
cov_1[1, 2] = rho_2_3*sigma_2*sigma_3
normal_3d = torch.distributions.multivariate_normal.MultivariateNormal(mean, covariance_matrix=cov_1)

y = torch.Tensor([[1, 2, 3]])
torch.exp(normal_3d.log_prob(y))

In [ ]:
import numpy as np
from mayavi import mlab

s = np.zeros([21, 21, 21])
for i in range(21):
    for j in range(21):
        for k in range(21):
            s[i, j, k] = torch.exp(normal_3d.log_prob(torch.FloatTensor([i-10, j-10, k-10])))
            
src = mlab.pipeline.scalar_field(s)
mlab.pipeline.iso_surface(src, contours=[s.min()+0.1*s.ptp(), ], opacity=0.3)
mlab.pipeline.iso_surface(src, contours=[s.max()-0.1*s.ptp(), ],)

In [ ]:
x = np.linspace(0, 10, 1001)
y = np.linspace(0, 10, 1001)
z = np.linspace(0, 10, 1001)
x_p = x + 0.5 * np.random.randn(1001)
y_p = y + 0.5 * np.random.randn(1001)
z_p = z + 0.5 * np.random.randn(1001)

In [ ]:
ax = plt.axes(projection='3d')
ax.scatter(x, y, z)
ax.scatter(x_p, y_p, z_p)

In [ ]:
class MDN(nn.Module):
    def __init__(self, n_input, n_hidden, n_gaussians):
        super(MDN, self).__init__()
        self.l_h = nn.Sequential(
            nn.Linear(n_input, n_hidden),
            nn.Tanh()
        )
        self.l_pi = nn.Linear(n_hidden, n_gaussians)
        
        self.l_mu_x = nn.Linear(n_hidden, n_gaussians)
        self.l_sigma_x = nn.Linear(n_hidden, n_gaussians)
        
        self.l_mu_y = nn.Linear(n_hidden, n_gaussians)
        self.l_sigma_y = nn.Linear(n_hidden, n_gaussians)
        
        self.l_mu_z = nn.Linear(n_hidden, n_gaussians)
        self.l_sigma_z = nn.Linear(n_hidden, n_gaussians)
        
        # self.l_correlation_x_y = nn.Linear(n_hidden, n_gaussians)
        # self.l_correlation_x_z = nn.Linear(n_hidden, n_gaussians)
        # self.l_correlation_y_z = nn.Linear(n_hidden, n_gaussians)
        
        self.l_correlation_x_y = nn.Sequential(
            nn.Linear(n_hidden, n_gaussians),
            nn.Tanh()
        )
        self.l_correlation_x_z = nn.Sequential(
            nn.Linear(n_hidden, n_gaussians),
            nn.Tanh()
        )
        self.l_correlation_y_z = nn.Sequential(
            nn.Linear(n_hidden, n_gaussians),
            nn.Tanh()
        )
    
    def forward(self, x):
        h = self.l_h(x)
        # print("h", h.shape)
        # print("h[0]", h[0, :])
        pi = F.softmax(self.l_pi(h), -1)
        # print("pi", pi.shape)
        # print("pi[0]", pi[0, :])
        mu_x = self.l_mu_x(h)
        # print("mu_x", pi.shape)
        mu_y = self.l_mu_y(h)
        # print("mu_y", pi.shape)
        mu_z = self.l_mu_z(h)
        # print("mu_z", pi.shape)
        
        # use exp to ensure positive range
        sigma_x = torch.exp(self.l_sigma_x(h))
        # print("sigma_x", pi.shape)
        sigma_y = torch.exp(self.l_sigma_y(h))
        # print("sigma_y", pi.shape)
        sigma_z = torch.exp(self.l_sigma_z(h))
        # print("sigma_z", pi.shape)
        
        # use tanh to ensoure range of (-1, 1)
        correlation_x_y = self.l_correlation_x_y(h)
        # print("correlation_x_y", pi.shape)
        # print("correlation_x_y[0]", correlation_x_y[0, :])
        correlation_x_z = self.l_correlation_x_z(h)
        # print("correlation_x_z", pi.shape)
        # print("correlation_x_z[0]", correlation_x_z[0, :])
        correlation_y_z = self.l_correlation_y_z(h)
        # print("correlation_y_z", pi.shape)
        # print("correlation_y_z[0]", correlation_y_z[0, :])
        
        return pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z, correlation_x_y, correlation_x_z, correlation_y_z

model = MDN(3, n_hidden=20, n_gaussians=5)
#if torch.cuda.is_available():
#    model.cuda()

optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def mdn_loss_fn(y, pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z, correlation_x_y, correlation_x_z, correlation_y_z):
    # print("mu_x shape", mu_x.shape)
    # mu = torch.stack((mu_x, mu_y, mu_z), 2)
    # print("mu shape ", mu.shape)
    # print("mu[0]", mu[0])
    
    ##############################################################################
    
    size = y.shape[0]
    n_gaussians = pi.shape[1]
    # print("sample size: ", size)
    # print("num of gaus: ", n_gaussians)
    # print("correlation size: ", correlation_x_y.shape)
    # build mean matrix
    mean = torch.zeros([size, n_gaussians, 3])
    # build covariance matrix with standard trivariate normal distribution
    cov = torch.ones([size, n_gaussians, 3, 3])
    cov[:, :, 0, 1] = correlation_x_y
    cov[:, :, 1, 0] = correlation_x_y
    cov[:, :, 0, 2] = correlation_x_z
    cov[:, :, 2, 0] = correlation_x_z
    cov[:, :, 1, 2] = correlation_y_z
    cov[:, :, 2, 1] = correlation_y_z
    
    print("mean: ", mean.shape)
    print("cov: ", cov.shape)
    
    print("mean is cuda: ", mean.is_cuda)
    print(" cov is cuda: ", cov.is_cuda)
    
    
    
    
    '''
    w = y[0]**2*(correlation_y_z**2 - 1) +\
        y[1]**2*(correlation_x_z**2 - 1) +\
        y[2]**2*(correlation_x_y**2 - 1) +\
        2*(y[0]*y[:1]*(correlation_x_y - correlation_x_z*correlation_y_z) +\
           y[0]*y[:2]*(correlation_x_z - correlation_x_y*correlation_y_z) +\
           y[1]*y[:2]*(correlation_y_z - correlation_x_y*correlation_x_z))
    
    return w
    '''
    '''
    cov = torch.Tensor([[sigma_x**2, 0, 0], [0, sigma_y**2, 0], [0, 0, sigma_z**2]])
    cov[1, 0] = correlation_x_y*sigma_x*sigma_y
    cov[2, 0] = correlation_x_z*sigma_x*sigma_z
    cov[2, 1] = correlation_y_z*sigma_y*sigma_z
    cov[0, 1] = correlation_x_y*sigma_x*sigma_y
    cov[0, 2] = correlation_x_z*sigma_x*sigma_z
    cov[1, 2] = correlation_y_z*sigma_y*sigma_z
    '''
    m = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)
    # normalize input y
    y_repeat = y.unsqueeze(1).repeat(1, n_gaussians, 1)
    # print(y_repeat.shape)
    # print(y_repeat[0])
    mu_all = torch.stack((mu_x, mu_y, mu_z), dim = 2)
    print("mu_all shape: ", mu_all.shape)
    y_sub = y_repeat - mu_all
    print("y_sub shape", y_sub.shape)
    sigma_all = torch.stack((sigma_x, sigma_y, sigma_z), dim = 2)
    print("sigma", sigma_x.shape)
    print("sigma all", sigma_all.shape)
    
    y_normal = torch.div(y_sub, sigma_all)
    
    print("y_normal size", y_normal.shape)
    
    # y: [x, y, z]; row: x, y, z; column: N, number of samples
    # y: N x 3
    # loss: N x 1
    print(y_normal.is_cuda)
    loss = torch.exp(m.log_prob(y_normal))
    print("loss shape: ", loss.shape)
    print("pi shape: ", pi.shape)
    loss = torch.sum(loss * pi, dim=1)
    loss = -torch.log(loss)
    return torch.mean(loss)

In [ ]:
######################################################333

In [ ]:
for epoch in range(1):
    pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z, correlation_x_y, correlation_x_z, correlation_y_z = model(pd_label_tensor)
    loss = mdn_loss_fn(gt_label_tensor, pi, mu_x, mu_y, mu_z, sigma_x, sigma_y, sigma_z, correlation_x_y, correlation_x_z, correlation_y_z)
    # optimizer.zero_grad()
    # loss.backward()
    # optimizer.step()
    # if epoch % 1000 == 0:
    #     print(loss.data.tolist())

In [ ]:
mean.shape

In [ ]:
t = torch.rand(2,3)
print(t)
tr = t.repeat(2, 3)
print(tr)

In [ ]:
a = torch.Tensor([[1,2],[4,5],[7,8]])

In [ ]:
b = torch.Tensor([[7,8],[6,4],[1,2]])

In [ ]:
a

In [ ]:
b

In [ ]:
a*b

In [ ]:
torch.sum(a*b, dim=1)

In [ ]:
tt[0,0,0] = 7

In [ ]:
tt

In [ ]:
t

In [ ]:
t[0, 0] = 7

In [ ]:
t

In [ ]:
tr

In [ ]:
correlation_y_z

In [ ]:
0.2939**2

In [ ]:
a

In [ ]:
torch.square(a)

In [ ]:
torch.square(d)

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(gt_label_tensor[:,0], gt_label_tensor[:,1], gt_label_tensor[:,2], 'gray')
ax.plot3D(pd_label_tensor[:,0], pd_label_tensor[:,1], pd_label_tensor[:,2], 'red')

In [ ]:
from mpl_toolkits import mplot3d
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(pd_label_tensor[:,0], pd_label_tensor[:,1], pd_label_tensor[:,2], 'gray')

In [ ]:
plt.plot(gt_label_tensor[:,2])
plt.plot(pd_label_tensor[:,2])

In [ ]:
a

In [ ]:
input_tensor

In [ ]:
test_1_loss = []
for i in range(1, 11):
    window_size = i
    PATH = "model/with_early_stop_after_400/model_w_" + str(i) + ".pt"

    test_set_x_1, test_set_y_1, test_set_z_1 = load_test_data('../../../../performance_test/data/test/test_1.csv')
    test_set_x_2, test_set_y_2, test_set_z_2 = load_test_data('../../../../performance_test/data/test/test_2.csv')
    test_set_x_3, test_set_y_3, test_set_z_3 = load_test_data('../../../../performance_test/data/test/test_3.csv')
    test_set_x_4, test_set_y_4, test_set_z_4 = load_test_data('../../../../performance_test/data/test/test_4.csv')
    test_set_x_5, test_set_y_5, test_set_z_5 = load_test_data('../../../../performance_test/data/test/test_5.csv')

    # do min-max-scaling for each test data set
    show_statistic(test_set_x_1)
    min_max_scaling(test_set_x_1)
    show_statistic(test_set_x_1)
    min_max_scaling(test_set_x_2)
    min_max_scaling(test_set_x_3)
    min_max_scaling(test_set_x_4)
    min_max_scaling(test_set_x_5)

    min_max_scaling(test_set_y_1)
    min_max_scaling(test_set_y_2)
    min_max_scaling(test_set_y_3)
    min_max_scaling(test_set_y_4)
    min_max_scaling(test_set_y_5)

    min_max_scaling(test_set_z_1)
    min_max_scaling(test_set_z_2)
    min_max_scaling(test_set_z_3)
    min_max_scaling(test_set_z_4)
    min_max_scaling(test_set_z_5)


    show_statistic(test_set_x_1)
    # do normalization on x of validation test set
    normalize_one(test_set_x_1, x_mean, x_std)
    show_statistic(test_set_x_1)
    normalize_one(test_set_x_2, x_mean, x_std)
    normalize_one(test_set_x_3, x_mean, x_std)
    normalize_one(test_set_x_4, x_mean, x_std)
    normalize_one(test_set_x_5, x_mean, x_std)
    # do normalization on y of validation test set
    normalize_one(test_set_y_1, y_mean, y_std)
    normalize_one(test_set_y_2, y_mean, y_std)
    normalize_one(test_set_y_3, y_mean, y_std)
    normalize_one(test_set_y_4, y_mean, y_std)
    normalize_one(test_set_y_5, y_mean, y_std)
    # do normalization on z of validation test set
    normalize_one(test_set_z_1, z_mean, z_std)
    normalize_one(test_set_z_2, z_mean, z_std)
    normalize_one(test_set_z_3, z_mean, z_std)
    normalize_one(test_set_z_4, z_mean, z_std)
    normalize_one(test_set_z_5, z_mean, z_std)
    # show_statistic(train_set_x_1)


    test_dataset_1, test_label_1 = construct_test_tensor(test_set_x_1,
                                                         test_set_y_1,
                                                         test_set_z_1,
                                                         window_size)
    test_dataset_2, test_label_2 = construct_test_tensor(test_set_x_2,
                                                         test_set_y_2,
                                                         test_set_z_2,
                                                         window_size)
    test_dataset_3, test_label_3 = construct_test_tensor(test_set_x_3,
                                                         test_set_y_3,
                                                         test_set_z_3,
                                                         window_size)
    test_dataset_4, test_label_4 = construct_test_tensor(test_set_x_4,
                                                         test_set_y_4,
                                                         test_set_z_4,
                                                         window_size)
    test_dataset_5, test_label_5 = construct_test_tensor(test_set_x_5,
                                                         test_set_y_5,
                                                         test_set_z_5,
                                                         window_size)

    test_set_1 = MyTestDataSet(test_dataset_1, test_label_1)
    test_set_2 = MyTestDataSet(test_dataset_2, test_label_2)
    test_set_3 = MyTestDataSet(test_dataset_3, test_label_3)
    test_set_4 = MyTestDataSet(test_dataset_4, test_label_4)
    test_set_5 = MyTestDataSet(test_dataset_5, test_label_5)
    print(len(test_set_1))
    print(len(test_set_2))
    print(len(test_set_3))
    print(len(test_set_4))
    print(len(test_set_5))
    show_statistic(test_set_x_1)
    show_statistic(test_set_x_2)
    show_statistic(test_set_x_3)
    show_statistic(test_set_x_4)
    show_statistic(test_set_x_5)


    batch_size = 650
    test_loader_1 = DataLoader(test_set_1, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_2 = DataLoader(test_set_2, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_3 = DataLoader(test_set_3, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_4 = DataLoader(test_set_4, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_5 = DataLoader(test_set_5, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    
    input_dim = 3
    hidden_dim = 100
    layer_dim = 1
    output_dim = 3

    load_model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
    load_model.load_state_dict(torch.load(PATH))
    load_model.eval()
    # mmodel = torch.load(PATH)

    criterion = nn.MSELoss()

    if torch.cuda.is_available():
        load_model.cuda()

    # get test results
    seq_dim = window_size
    input_dim = 3
    # test_seq = []
    test_predd = []
    # test_gt = []
    total_test_loss = 0.0
    test_batch = 0
    for i, (seqs, labels) in enumerate(test_loader_1):
        if torch.cuda.is_available():
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))

        outputs = load_model(seqs)
        loss = criterion(outputs, labels)
        total_test_loss += loss.data.item()
        test_predd.append(outputs)
        test_batch = i + 1

    print(total_test_loss/test_batch)
    test_1_loss.append(total_test_loss/te    input_dim = 3
    hidden_dim = 100
    layer_dim = 1
    output_dim = 3

    load_model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
    load_model.load_state_dict(torch.load(PATH))
    load_model.eval()
    # mmodel = torch.load(PATH)

    criterion = nn.MSELoss()

    if torch.cuda.is_available():
        load_model.cuda()

    # get test results
    seq_dim = window_size
    input_dim = 3
    # test_seq = []
    test_predd = []
    # test_gt = []
    total_test_loss = 0.0
    test_batch = 0
    for i, (seqs, labels) in enumerate(test_loader_1):
        if torch.cuda.is_available():
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))

        outputs = load_model(seqs)
        loss = criterion(outputs, labels)
        total_test_loss += loss.data.item()
        test_predd.append(outputs)
        test_batch = i + 1

    print(total_test_loss/test_batch)st_batch)

    for i in range(len(test_predd)):
        if (i == 0):
            pred = test_predd[i].cpu().detach().numpy()
        else:
            pred = np.append(pred, test_predd[i].cpu().detach().numpy(), axis = 0)

    from mpl_toolkits import mplot3d
    fig = plt.figure()
    ax = plt.axes(projection='3d')

    # Data for a three-dimensional line
    zline = np.linspace(0, 15, 1000)
    xline = np.sin(zline)
    yline = np.cos(zline)
    ax.plot3D(test_set_x_1, test_set_y_1, test_set_z_1, 'gray')
    ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')

In [ ]:
test_1_loss

In [ ]:
#find window size 6 is optimal

In [ ]:
x = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

x_pos = [i for i, _ in enumerate(x)]

# plt.bar(x_pos, test_1_loss)
plt.bar(x_pos, test_1_loss, color=['tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:green',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue'], zorder = 3)

plt.grid(zorder=0)

plt.xlabel("Input window size", fontsize=14)
plt.ylabel("MSE loss", fontsize=14)
plt.xticks(x_pos, x)

plt.ylim([0.017,0.0255])
# plt.show()

Plot result without normalization

In [ ]:
test_3_loss = []
for i in range(1, 11):
    window_size = i
    PATH = "model_v3/with_early_stop_after_400_without_normalization/model_w_" + str(i) + ".pt"

    test_set_x_1, test_set_y_1, test_set_z_1 = load_test_data('../../../../performance_test/data/test/test_1.csv')
    test_set_x_2, test_set_y_2, test_set_z_2 = load_test_data('../../../../performance_test/data/test/test_2.csv')
    test_set_x_3, test_set_y_3, test_set_z_3 = load_test_data('../../../../performance_test/data/test/test_3.csv')
    test_set_x_4, test_set_y_4, test_set_z_4 = load_test_data('../../../../performance_test/data/test/test_4.csv')
    test_set_x_5, test_set_y_5, test_set_z_5 = load_test_data('../../../../performance_test/data/test/test_5.csv')

    '''
    # do min-max-scaling for each test data set
    show_statistic(test_set_x_1)
    min_max_scaling(test_set_x_1)
    show_statistic(test_set_x_1)
    min_max_scaling(test_set_x_2)
    min_max_scaling(test_set_x_3)
    min_max_scaling(test_set_x_4)
    min_max_scaling(test_set_x_5)

    min_max_scaling(test_set_y_1)
    min_max_scaling(test_set_y_2)
    min_max_scaling(test_set_y_3)
    min_max_scaling(test_set_y_4)
    min_max_scaling(test_set_y_5)

    min_max_scaling(test_set_z_1)
    min_max_scaling(test_set_z_2)
    min_max_scaling(test_set_z_3)
    min_max_scaling(test_set_z_4)
    min_max_scaling(test_set_z_5)
    '''
    '''
    show_statistic(test_set_x_1)
    # do normalization on x of validation test set
    normalize_one(test_set_x_1, x_mean, x_std)
    show_statistic(test_set_x_1)
    normalize_one(test_set_x_2, x_mean, x_std)
    normalize_one(test_set_x_3, x_mean, x_std)
    normalize_one(test_set_x_4, x_mean, x_std)
    normalize_one(test_set_x_5, x_mean, x_std)
    # do normalization on y of validation test set
    normalize_one(test_set_y_1, y_mean, y_std)
    normalize_one(test_set_y_2, y_mean, y_std)
    normalize_one(test_set_y_3, y_mean, y_std)
    normalize_one(test_set_y_4, y_mean, y_std)
    normalize_one(test_set_y_5, y_mean, y_std)
    # do normalization on z of validation test set
    normalize_one(test_set_z_1, z_mean, z_std)
    normalize_one(test_set_z_2, z_mean, z_std)
    normalize_one(test_set_z_3, z_mean, z_std)
    normalize_one(test_set_z_4, z_mean, z_std)
    normalize_one(test_set_z_5, z_mean, z_std)
    # show_statistic(train_set_x_1)
    '''

    test_dataset_1, test_label_1 = construct_test_tensor(test_set_x_1,
                                                         test_set_y_1,
                                                         test_set_z_1,
                                                         window_size)
    test_dataset_2, test_label_2 = construct_test_tensor(test_set_x_2,
                                                         test_set_y_2,
                                                         test_set_z_2,
                                                         window_size)
    test_dataset_3, test_label_3 = construct_test_tensor(test_set_x_3,
                                                         test_set_y_3,
                                                         test_set_z_3,
                                                         window_size)
    test_dataset_4, test_label_4 = construct_test_tensor(test_set_x_4,
                                                         test_set_y_4,
                                                         test_set_z_4,
                                                         window_size)
    test_dataset_5, test_label_5 = construct_test_tensor(test_set_x_5,
                                                         test_set_y_5,
                                                         test_set_z_5,
                                                         window_size)

    test_set_1 = MyTestDataSet(test_dataset_1, test_label_1)
    test_set_2 = MyTestDataSet(test_dataset_2, test_label_2)
    test_set_3 = MyTestDataSet(test_dataset_3, test_label_3)
    test_set_4 = MyTestDataSet(test_dataset_4, test_label_4)
    test_set_5 = MyTestDataSet(test_dataset_5, test_label_5)
    print(len(test_set_1))
    print(len(test_set_2))
    print(len(test_set_3))
    print(len(test_set_4))
    print(len(test_set_5))
    show_statistic(test_set_x_1)
    show_statistic(test_set_x_2)
    show_statistic(test_set_x_3)
    show_statistic(test_set_x_4)
    show_statistic(test_set_x_5)


    batch_size = 650
    test_loader_1 = DataLoader(test_set_1, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_2 = DataLoader(test_set_2, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_3 = DataLoader(test_set_3, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_4 = DataLoader(test_set_4, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader_5 = DataLoader(test_set_5, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
    # test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
    
    input_dim = 3
    hidden_dim = 100
    layer_dim = 1
    output_dim = 3

    load_model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
    load_model.load_state_dict(torch.load(PATH))
    load_model.eval()
    # mmodel = torch.load(PATH)

    criterion = nn.MSELoss()

    if torch.cuda.is_available():
        load_model.cuda()

    # get test results
    seq_dim = window_size
    input_dim = 3
    # test_seq = []
    test_predd = []
    # test_gt = []
    total_test_loss = 0.0
    test_batch = 0
    for i, (seqs, labels) in enumerate(test_loader_3):
        if torch.cuda.is_available():
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))

        outputs = load_model(seqs)
        loss = criterion(outputs, labels)
        total_test_loss += loss.data.item()
        test_predd.append(outputs)
        test_batch = i + 1

    print(total_test_loss/test_batch)
    test_3_loss.append(total_test_loss/test_batch)

    for i in range(len(test_predd)):
        if (i == 0):
            pred = test_predd[i].cpu().detach().numpy()
        else:
            pred = np.append(pred, test_predd[i].cpu().detach().numpy(), axis = 0)

    from mpl_toolkits import mplot3d
    fig = plt.figure()
    ax = plt.axes(projection='3d')

    # Data for a three-dimensional line
    zline = np.linspace(0, 15, 1000)
    xline = np.sin(zline)
    yline = np.cos(zline)
    ax.plot3D(test_set_x_3, test_set_y_3, test_set_z_3, 'gray')
    ax.plot3D(pred[:,0], pred[:,1], pred[:,2], 'red')

In [ ]:
test_3_loss

In [ ]:
min(test_3_loss)

In [ ]:
x = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

x_pos = [i for i, _ in enumerate(x)]

# plt.bar(x_pos, test_1_loss)
plt.bar(x_pos, test_3_loss, color=['tab:blue',
                                   'tab:blue',
                                   'tab:green',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue',
                                   'tab:blue'], zorder = 3)

plt.grid(zorder=0)

plt.xlabel("Input Window Size", fontsize=12)
plt.ylabel("MSE Loss", fontsize=12)
plt.xticks(x_pos, x)

# plt.ylim([0.017,0.0255])
plt.ylim([0.025,0.039])
# plt.show()